# Model Comparison Analysis

This notebook provides comprehensive analysis and comparison of different ML models for earthquake prediction.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.append(str(Path().parent / 'src'))

from prediction_engine import EarthquakePredictionEngine
from utils.model_evaluator import EarthquakeModelEvaluator
from utils.visualization import EarthquakeVisualization
from models.random_forest_model import EnhancedRandomForestModel
from models.xgboost_model import XGBoostModel
from models.neural_network_model import EnhancedNeuralNetworkModel
from models.svm_model import SVMEarthquakeModel
from models.ensemble_model import EarthquakeEnsembleModel

# Set style
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
sns.set_palette("husl")

%matplotlib inline

## 1. Data Preparation

In [ ]:
# Initialize the prediction engine
engine = EarthquakePredictionEngine()

# Collect and prepare data
print("Collecting earthquake data...")
raw_data = engine.data_collector.fetch_enhanced_data(days_back=180, min_magnitude=4.0)
print(f"Collected {len(raw_data)} earthquake records")

# Feature engineering
print("Engineering features...")
feature_data = engine.feature_engineer.create_all_features(raw_data)
print(f"Created {len(feature_data.columns)} features")

# Preprocessing
X, y = engine.data_preprocessor.fit_transform(feature_data, target_col='magnitude')
X_train, X_test, y_train, y_test = engine.data_preprocessor.create_train_test_split(X, y, temporal_split=False)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Features: {X_train.shape[1]}")

## 2. Individual Model Training and Evaluation

In [ ]:
# Initialize evaluator
evaluator = EarthquakeModelEvaluator()

# Dictionary to store all results
model_results = {}

# Get feature names for analysis
feature_names = [f'feature_{i}' for i in range(X_train.shape[1])]

### Random Forest Model

In [ ]:
print("Training Random Forest...")
rf_model = EnhancedRandomForestModel()
rf_training_results = rf_model.train(X_train, y_train, optimize_params=False)

# Evaluate
rf_results = evaluator.evaluate_model(
    rf_model, X_train, X_test, y_train, y_test, 
    model_name="Random Forest", feature_names=feature_names
)

model_results['Random Forest'] = rf_results

print(f"Random Forest R²: {rf_results['test_metrics']['r2']:.4f}")

### XGBoost Model

In [ ]:
print("Training XGBoost...")
xgb_model = XGBoostModel()
xgb_training_results = xgb_model.train(X_train, y_train, optimize_params=False)

# Evaluate
xgb_results = evaluator.evaluate_model(
    xgb_model, X_train, X_test, y_train, y_test, 
    model_name="XGBoost", feature_names=feature_names
)

model_results['XGBoost'] = xgb_results

print(f"XGBoost R²: {xgb_results['test_metrics']['r2']:.4f}")

### SVM Model

In [ ]:
print("Training SVM...")
svm_model = SVMEarthquakeModel()
svm_training_results = svm_model.train(X_train, y_train, optimize_params=False)

# Evaluate
svm_results = evaluator.evaluate_model(
    svm_model, X_train, X_test, y_train, y_test, 
    model_name="SVM"
)

model_results['SVM'] = svm_results

print(f"SVM R²: {svm_results['test_metrics']['r2']:.4f}")

### Ensemble Model

In [ ]:
print("Training Ensemble...")
ensemble_model = EarthquakeEnsembleModel()
ensemble_model.initialize_models(['random_forest', 'xgboost', 'svm'])

ensemble_training_results = ensemble_model.train(
    X_train, y_train,
    ensemble_type='weighted_average',
    optimize_weights=True
)

# Evaluate
ensemble_results = evaluator.evaluate_model(
    ensemble_model, X_train, X_test, y_train, y_test, 
    model_name="Ensemble"
)

model_results['Ensemble'] = ensemble_results

print(f"Ensemble R²: {ensemble_results['test_metrics']['r2']:.4f}")
print(f"Ensemble weights: {ensemble_training_results['ensemble_weights']}")

## 3. Model Comparison Analysis

In [ ]:
# Generate model comparison
comparison = evaluator.compare_models(model_results)

print("Model Comparison Results:")
print("=" * 40)

if 'overall_ranking' in comparison:
    print("Overall Ranking (by Test R²):")
    for i, model_info in enumerate(comparison['overall_ranking']):
        print(f"{i+1}. {model_info['model']}: R² = {model_info['score']:.4f}")

print("\nDetailed Metrics:")
for model_name, results in model_results.items():
    if 'error' not in results:
        metrics = results['test_metrics']
        print(f"{model_name}:")
        print(f"  R²: {metrics['r2']:.4f}")
        print(f"  RMSE: {metrics['rmse']:.4f}")
        print(f"  MAE: {metrics['mae']:.4f}")

## 4. Visualization

In [ ]:
# Initialize visualizer
visualizer = EarthquakeVisualization()

# Model comparison plot
valid_results = {k: v for k, v in model_results.items() if 'error' not in v}
if len(valid_results) >= 2:
    comparison_fig = visualizer.plot_model_comparison(valid_results)
    plt.show()
else:
    print("Need at least 2 valid models for comparison plot")

## 5. Feature Importance Analysis

In [ ]:
# Analyze feature importance for tree-based models
if 'Random Forest' in model_results and 'feature_importance' in model_results['Random Forest']:
    rf_importance = model_results['Random Forest']['feature_importance']['top_10_features']
    rf_importance_df = pd.DataFrame(rf_importance)
    
    # Plot feature importance
    importance_fig = visualizer.plot_feature_importance(rf_importance_df, top_n=15)
    plt.show()
    
    print("Top 10 Most Important Features (Random Forest):")
    for i, feat in enumerate(rf_importance[:10]):
        print(f"{i+1:2d}. {feat['feature']:<25} {feat['importance']:.4f}")

## 6. Prediction Analysis

In [ ]:
# Get predictions from the best model
best_model_name = None
best_r2 = -999

for name, results in model_results.items():
    if 'error' not in results:
        r2 = results['test_metrics']['r2']
        if r2 > best_r2:
            best_r2 = r2
            best_model_name = name

if best_model_name:
    print(f"Best model: {best_model_name} (R² = {best_r2:.4f})")
    
    # Get the best model object
    if best_model_name == 'Random Forest':
        best_model = rf_model
    elif best_model_name == 'XGBoost':
        best_model = xgb_model
    elif best_model_name == 'SVM':
        best_model = svm_model
    elif best_model_name == 'Ensemble':
        best_model = ensemble_model
    
    # Make predictions
    y_pred = best_model.predict(X_test)
    
    # Get uncertainties if available
    uncertainties = None
    if hasattr(best_model, 'predict_with_uncertainty'):
        _, uncertainties = best_model.predict_with_uncertainty(X_test)
    
    # Plot prediction results
    pred_fig = visualizer.plot_prediction_results(
        y_test, y_pred, uncertainties, model_name=best_model_name
    )
    plt.show()

## 7. Model Performance Summary

In [ ]:
# Create summary table
summary_data = []

for model_name, results in model_results.items():
    if 'error' not in results:
        metrics = results['test_metrics']
        cv_results = results.get('cv_results', {})
        
        summary_data.append({
            'Model': model_name,
            'Test R²': f"{metrics['r2']:.4f}",
            'Test RMSE': f"{metrics['rmse']:.4f}",
            'Test MAE': f"{metrics['mae']:.4f}",
            'CV R² Mean': f"{cv_results.get('r2', {}).get('mean', 0):.4f}",
            'CV R² Std': f"{cv_results.get('r2', {}).get('std', 0):.4f}",
            'Performance': results.get('performance_summary', {}).get('performance_category', 'unknown').title()
        })

summary_df = pd.DataFrame(summary_data)
print("\nModel Performance Summary:")
print("=" * 80)
print(summary_df.to_string(index=False))

## 8. Recommendations

In [ ]:
print("\n" + "=" * 60)
print("RECOMMENDATIONS")
print("=" * 60)

if best_model_name:
    print(f"\n1. BEST PERFORMING MODEL: {best_model_name}")
    print(f"   - Test R²: {best_r2:.4f}")
    
    if best_r2 >= 0.9:
        print("   - EXCELLENT performance - ready for production")
    elif best_r2 >= 0.8:
        print("   - GOOD performance - suitable for most applications")
    elif best_r2 >= 0.6:
        print("   - FAIR performance - may need improvement")
    else:
        print("   - POOR performance - requires significant improvement")

# Check if ensemble is best
if 'Ensemble' in model_results and 'error' not in model_results['Ensemble']:
    ensemble_r2 = model_results['Ensemble']['test_metrics']['r2']
    print(f"\n2. ENSEMBLE MODEL PERFORMANCE:")
    print(f"   - R²: {ensemble_r2:.4f}")
    
    # Compare with individual models
    individual_r2s = []
    for name, results in model_results.items():
        if name != 'Ensemble' and 'error' not in results:
            individual_r2s.append(results['test_metrics']['r2'])
    
    if individual_r2s:
        max_individual_r2 = max(individual_r2s)
        if ensemble_r2 > max_individual_r2:
            print("   - Ensemble OUTPERFORMS individual models")
            print("   - RECOMMENDATION: Use ensemble for production")
        else:
            print("   - Individual models perform better")
            print("   - RECOMMENDATION: Use best individual model")

print(f"\n3. UNCERTAINTY ESTIMATION:")
uncertainty_models = []
for name, results in model_results.items():
    if 'uncertainty_metrics' in results:
        uncertainty_models.append(name)

if uncertainty_models:
    print(f"   - Models with uncertainty: {', '.join(uncertainty_models)}")
    print("   - RECOMMENDATION: Use uncertainty for risk assessment")
else:
    print("   - No uncertainty metrics available")
    print("   - RECOMMENDATION: Implement uncertainty estimation")

print(f"\n4. FEATURE IMPORTANCE:")
if 'Random Forest' in model_results and 'feature_importance' in model_results['Random Forest']:
    top_features = model_results['Random Forest']['feature_importance']['top_10_features'][:3]
    print("   - Top 3 most important features:")
    for i, feat in enumerate(top_features):
        print(f"     {i+1}. {feat['feature']} ({feat['importance']:.4f})")
    print("   - RECOMMENDATION: Focus on these features for model interpretation")

print(f"\n5. DEPLOYMENT STRATEGY:")
print("   - Use ensemble model for robustness")
print("   - Implement real-time monitoring")
print("   - Set up model retraining pipeline")
print("   - Include uncertainty quantification in alerts")